<a href="https://colab.research.google.com/github/Arunachalam-J/Python-restapis/blob/main/Rag_with_RetrieverQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain faiss-gpu sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=3fa9a73f75391f71b6524e941a9a5051319b5e58e6580d703249c8322d192824
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence

In [2]:
from langchain.prompts import ChatPromptTemplate
# from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [3]:
embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [4]:
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from bs4 import BeautifulSoup as Soup


In [41]:
def get_text(html_text):
    parsed = Soup(html_text, "html.parser")
    data = parsed.find('div', class_='document')
    return data

In [45]:
url = "https://docs.python.org/3.9/"
loader = RecursiveUrlLoader(
    url=url, max_depth=2, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

In [ ]:
docs[0]

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [8]:
doc_splitter = CharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=500,
    chunk_overlap=20,
)

In [ ]:
document = doc_splitter.split_documents(docs)

In [10]:
len(document)

10160

In [11]:
f_db = FAISS.from_documents(document, embedding)

In [12]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [13]:
from langchain.llms import HuggingFacePipeline

In [28]:
import torch
torch.cuda.device_count()

1

In [29]:
hf = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    device=0,
    pipeline_kwargs={"max_new_tokens": 100, "top_p":0.92,"temperature":0.5},
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# text_db = FAISS.from_texts(["Python is a great language","python is a scripting language"],embedding)

In [36]:
retreiver = f_db.as_retriever(search_kwargs={"k": 6})

In [18]:
retreiver.get_relevant_documents("what is in new in python 3.9?")

[Document(page_content='previous |\n\nPython »\n\n3.9.17 Documentation »\n    \nWhatâ\x80\x99s New in Python »\n\n                     |\n                \n\nWhatâ\x80\x99s New In Python 3.8Â¶\n\nEditor\nRaymond Hettinger\n\n\nThis article explains the new features in Python 3.8, compared to 3.7.\nFor full details, see the changelog.\n\nSummary â\x80\x93 Release highlightsÂ¶\n\n\nNew FeaturesÂ¶', metadata={'source': 'https://docs.python.org/3.9/whatsnew/3.8.html', 'title': 'Whatâ\x80\x99s New In Python 3.8 — Python 3.9.17 documentation', 'language': None}),
 Document(page_content='|\n                \n\n\nPython 3.9.17 documentation\n\n  Welcome! This is the official documentation for Python 3.9.17.\n  \nParts of the documentation:\n\n\nWhat\'s new in Python 3.9?\n or all "What\'s new" documents since 2.0\nTutorial\nstart here\nLibrary Reference\nkeep this under your pillow\nLanguage Reference\ndescribes syntax and language elements\nPython Setup and Usage\nhow to use Python on differe

In [19]:
from langchain.chains import RetrievalQA

In [37]:
qa = RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retreiver,
    chain_type_kwargs = {"prompt": prompt})

In [21]:
from langchain.prompts import PromptTemplate

In [22]:
prompt_template = """
As a helpful energy specialist, please answer the question below, focusing on numerical data and using only the context below.
Don't invent facts. If you can't provide a factual answer, say you don't know what the answer is.

question: {question}

context: {context}
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [38]:
question = " what is python 3.9 ?"
answer = qa.run({"query":question})
print(answer)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprec


|

What's new in Python 3.9?

or all "What's new" documents since 2.0

Tutorial

start here

Library Reference

keep this under your pillow

Language Reference

describes syntax and language elements

Python Setup and Usage

how to use Python on different platforms

Python HOWTOs

in-depth documents on specific topics

|

What's new in Python 3.
